In [ ]:
import itertools as it

import numpy as np
import pandas as pd
from scipy import optimize, sparse

from linprog import resize, variable

π = pd.read_csv("data/portfolio.csv", index_col=0)
M = len(π)
N = 2  # start with 2 brokerage accounts

# Margin minimization

Let us use [Linear Programming](https://en.wikipedia.org/wiki/Linear_programming) to allocate a 
[long/short equity](https://en.wikipedia.org/wiki/Long/short_equity) portfolio $\pi$ containing $M$
stocks between $N$ accounts with different [margin terms](https://en.wikipedia.org/wiki/Margin_(finance)) 
so as to _minimize_ total margin.

We introduce a vector $x$ of $M \cdot N$ unknowns to represent how much of stock $s$ sits in account 
$a$ (in $\$$).  Values of $x$ are naturally bounded by the total amount of stock we need to allocate.
Say the portfolio holds $\$100$ of stock $s$.  No account can hold less than $\$0$ or more than $\$100$.
Similarly, if the portfolio is [_short_](https://en.wikipedia.org/wiki/Short_(finance)) $\$80$ of $t$,
the corresponding ($N$) entries of $x$ belong in the $[-80, 0]$
[interval](https://en.wikipedia.org/wiki/Interval_(mathematics)).  $x$ is arranged in
[**row-major order**](https://en.wikipedia.org/wiki/Row-_and_column-major_order), as is usual
in [C](https://en.wikipedia.org/wiki/C_(programming_language)).  This means that $(s, a)$,
stock $s$ in account $a$ sits in entry $k := s + M \cdot a$.  This is precisely what
[`np.ravel`](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html) does.

This representation has two immediate advantages:
1. it precludes _boxes_ (allocating $\$100$ as $\$120 + \$(20)$)
1. it requires no _helpers_ (more on that later) for absolute values with stock granularity

This choice of representation makes the first [constraint](https://en.wikipedia.org/wiki/Constraint_(mathematics))
obvious: stock _allocations_ ($x$) must sum up to the entire portfolio.  This will introduce the
first necessary bit of [Linear Algebra](https://en.wikipedia.org/wiki/Linear_algebra).  Although $x$
is a (flat) vector, we can also think of it as a $N \times M$ matrix $X$.  Adding up stocks in different
accounts together amounts to $\mathbb{1}^T X$.  The matrix which achieves the same operation on $x$ is
a [horizontal concatenation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.hstack.html) of
[identity matrices](https://en.wikipedia.org/wiki/Identity_matrix).  We can use
use a [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) to define it.
This description makes it clear that we are dealing with
very [sparse](https://en.wikipedia.org/wiki/Sparse_matrix) matrices.  Fortunately, 
[SciPy](https://scipy.org/) has excellent [support](https://docs.scipy.org/doc/scipy/reference/sparse.html)
for sparse matrices:

In [ ]:
x = variable("x", M * N)
A_eq = sparse.kron(np.ones((1, N)), sparse.eye_array(M), format="csr")
b_eq = (π["Shares"] * π["Price"]).to_numpy()
sign = np.sign(π["Shares"]).to_numpy()

A lot is happening in this snippet.  First of all, to keep this example short (at the expense
of some complexity), we are _not_ using an [Algebraic Modeling Language](https://en.wikipedia.org/wiki/Algebraic_modeling_language)
like [AMPL](https://en.wikipedia.org/wiki/AMPL) or [CVXPY](https://www.cvxpy.org/) but working
directly in [Standard Form](https://en.wikipedia.org/wiki/Linear_programming#Standard_form) instead.
We follow SciPy's [linprog](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)'s
notation, namely:
$$\begin{eqnarray}
\min_x \ & c^T x \\
\mbox{such that} \ & A_{ub} x \leq b_{ub},\\
& A_{eq} x = b_{eq},\\
& l \leq x \leq u ,
\end{eqnarray}$$

As such, _equality_ constraints correspond to _rows_ in $A_{eq}$ and $b_{eq}$ and similarly for
_inequality_ and $A_{ub}$ and $b_{ub}$.  Note also how we picked the
[compressed sparse row](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format))
format for our sparse matrices.  This format allows fast matrix-vector multiplications which helps the
solver (although SciPy translates between sparse formats seamlessly).  Finally, we expect these to be the only
_equality_ constraints so produce a matrix directly.  For convenience, we will declare _inequality_ contraints
incrementally.  We can do that by keeping intermediate steps in a python
[`list`](https://docs.python.org/3/library/stdtypes.html#list) which we
[stack "vertically"](https://docs.scipy.org/doc/scipy-1.16.2/reference/generated/scipy.sparse.vstack.html)
at the end.  The `variable` helper implicitly keeps track of the increasing width of matrices that represent
variables.  This means we need to pad "narrower" matrices to combine them with "wider" ones.  Fortunately,
sparse matrices decouple their physical and logical sizes.  This makes it easy to
[resize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.resize.html)
them.

In [ ]:
A_ub = []
b_ub = []

## Base rate

Margin terms for stock portfolios often start with a simple base rate, between $5$ and $7.5\%$ of 
Gross Market Value.  This maps easily to the $c^T x$ objective term in standard form.  To keep this
example simple, we model a first account with a low (teaser?) base rate and several addons, whereas
the second account charges a higher base but no addons.

In [ ]:
c = [0.05 * sign, 0.06 * sign]

## Penalties and epigraphs

### Liquidity

Brokers often include _add-ons_ or _penalties_ to discourage illiquid portfolio.  The simplest form of
penalties compare single stock holdings to external observables.  For example, a broker might not like single
stock holdings larger then one day of [average traded volume](https://en.wikipedia.org/wiki/Volume_(finance)).
They could simply forbid illiquid holdings but they typically prefer to charge an additional (sometimes
punitive) fee on the portion of holdings in _excess_ of the threshold.  That excess can be handled a little
like [slack variables](https://en.wikipedia.org/wiki/Slack_variable): by introducing extra variables with
constraints that encode their semantics.  So, if $y$ represents "holdings over one day of average volume",
the objective function become $c^T x + d^T y$ where $c$ coefficients are the product of a (broker-specific)
base rate and the applicable stock price, while $d$ coefficients reflect the concentration fee.  $y$ is
subject to $x - \text{ADV} \leq y$ or, equivalently, $x - y \leq \text{ADV}$ to fit in $A_{ub}$ and $b_{ub}$.
As long as $d$ coefficients are _positive_, minimizing the problem will naturally _collapse_ $y$ to
$\max (x - \text{ADV}, 0)$.  Such helpers are called [epigraphs](https://en.wikipedia.org/wiki/Epigraph_(mathematics)).

In [ ]:
y = variable("(|x0| - ADV)₊", M)
x0 = sparse.eye_array(M, x.shape[0]) @ x
resize(x0, y)
c.append(np.full(M, .2))
A_ub.append(sparse.diags_array(sign) @ x0 - y)
b_ub.append(π["Volume"] * π["Price"])

### Concentration

The next type of penalty in terms of complexity compare single stock holdings to the overall portfolio.
Unlike _liquidity_ penalties where the thresholds were external values, _concentration_ thresholds are
a percentage of the account's gross, or net, market value.  Calculating account market values is complementary
to adding stocks in different accounts together.  In matrix form, this is $X \mathbb{1}$.  The matrix which
achieves the same operation on $x$ is block-diagonal with $1 \times N$ $\mathbb{1}$ on its diagonal.  We
can construct it as a [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product).  The constraints
which define $y$ are also slightly different: all terms contain unknowns and must therefore appear on the
left-hand side ($A_{ub}$) of the comparison ($x - \alpha G \leq y \implies x - \alpha G - y \leq 0$).

This introduces a new challenge: concentration penalties compare a vector of size $M$ and a _scalar_.
Numpy hides this complexity because it [broadcasts](https://numpy.org/doc/stable/user/basics.broadcasting.html)
implicitly.  The sparse matrices which represent linear variables and expressions require _explicit_
broadcasting.  We will only handle the special case of an operation between an $m \times n$ matrix
and a $1 \times n$ one (representing a scalar expression).  That operation requires multiplying the
scalar by $\mathbb{1}$ on the left-hand side.

In [ ]:
gmv = sparse.kron(sparse.eye_array(N), sign[np.newaxis, :], format="csr")
broadcast = sparse.csr_array(np.ones((M, 1)) @ gmv[:1, :])

y = variable("(|x0| - 5% G)₊", M)
resize(x0, broadcast, y)
c.append(np.full(M, 0.1))
A_ub.append(sparse.diags_array(sign) @ x0 - 0.05 * broadcast - y)
b_ub.append(np.zeros(M))

### Sector

Penalties up to now have single stock granularity.  This omits case where no single stock holding is particularly
large but several _similar_ (i.e. [correlated](https://en.wikipedia.org/wiki/Correlation)) stocks make up a
concerning fraction of the overall portfolio.  Prime brokers traditionally rely on simple proxies like industry 
or geography to capture these effects.  This implies we need a way to aggregate $x$ by some external label.  That
aggregation, or grouping, can be achieved by a [grouping matrix](https://themosekblog.blogspot.com/2020/05/grouping-in-fusion.html).

In [ ]:
keys, inverse = np.unique(π["Sector"], return_inverse=True)
K = len(keys)
sector_mv = sparse.csr_array((np.ones(shape=M), (inverse, np.arange(M, dtype=int))), shape=(K, M)) @ x0

s = variable("|sector|", K)
c.append(np.full(K, 1e-6))  # ε so epigraph converges
resize(sector_mv, s)
# s = |t| ⇒ t - s ≤ 0 ∧ -t - s ≤ 0
A_ub.append(sector_mv - s)
A_ub.append(-sector_mv - s)
b_ub.append(np.zeros(2 * K))

y = variable("(|sector| - 20% G)₊", K)
broadcast = sparse.csr_array(np.ones((K, 1)) @ gmv[:1, :])

resize(s, broadcast, y)
c.append(np.full(K, 0.2))
A_ub.append(s - 0.2 * broadcast - y)
b_ub.append(np.zeros(K))

### Directionality

Finally, brokers worry about extreme market events where [$\beta$](https://en.wikipedia.org/wiki/Beta_(finance))
dominates stock movements.  A crude way to manage this risk is to penalize _directional_ portfolios as measured
by $N/G$ or, similarly, $S/L$.  Note the following relationships:
$$\begin{eqnarray}
\begin{bmatrix} G \\ S \end{bmatrix} =
\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
\begin{bmatrix} L \\ S \end{bmatrix}
\end{eqnarray}$$
Or, equivalently,
$$\begin{eqnarray}
\begin{bmatrix} L \\ S \end{bmatrix} =
\frac{1}{2}
\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
\begin{bmatrix} G \\ N \end{bmatrix}
\end{eqnarray}$$
Finally, we can also create labels based on the _sign_ of holdings (`np.sign(π["Shares"]).map({1: "Long", -1: "Short"})`) and leverage `sumby`

In [ ]:
resize(*A_ub)

K = sum(variable.__defaults__[0].values())
A_eq.resize((A_eq.shape[0], K))

bounds = np.repeat([[0, np.inf]], K, axis=0)
bounds[:N * M, 0] = np.tile(np.minimum(b_eq, 0), N)
bounds[:N * M, 1] = np.tile(np.maximum(b_eq, 0), N)

solution = optimize.linprog(
    c=np.concatenate(c),
    A_ub=sparse.vstack(A_ub, format="csr"),
    b_ub=np.concatenate(b_ub),
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
)
assert solution.success
π.loc[:, "MV"] = π["Shares"] * π["Price"]
π.loc[:, "Account 0"] = solution.x[:M]
π.loc[:, "Account 1"] = solution.x[M:2 * M]
π

`solution.fun` is the numerical value of the objective function, i.e. `solution.x.dot(c)`.
This lets us attribute its contributions from the helper variables we introduced incrementally.

In [ ]:
variables = variable.__defaults__[0]
spans = dict(zip(variables, it.starmap(slice, it.pairwise(it.accumulate(variables.values(), initial=0)))))
w = np.concatenate(c)
pd.Series({
    key: solution.x[span].dot(w[span])
    for key, span in spans.items()
})

We can re-derive those intermediate results from $x$ to verify our constraints captured the proper semantics

In [ ]:
pd.Series({
    "x": 0.05 * π["Account 0"].abs().sum() + 0.06 * π["Account 1"].abs().sum(),
    "|x0| - ADV)₊": 0.2 * np.maximum(π["Account 0"].abs() - π["Volume"] * π["Price"], 0).sum(),
    "(|x0| - 5% G)₊": 0.1 * np.maximum(π["Account 0"].abs() - 0.05 * π["Account 0"].abs().sum(), 0).sum(),
    "(|sector| - 20% G)₊": 0.2 * np.maximum(π.groupby("Sector")["Account 0"].sum().abs() - 0.2 * π["Account 0"].abs().sum(), 0).sum(),
})

In [ ]:
π.groupby("Sector")[["Account 0"]].sum().abs().assign(Epigraph=solution.x[spans["|sector|"]])